# Linear Discriminant Analyisis #

### LDA Description ### 
Supervised learning technique. 

Project a feature space (n-dimensional samples) onto a smaller sub-space k ( where k $\leq$ n-1 ) while maiting the class discriminatory information. 

#### Assumptions ####
- Normally distributed data 
- Indentical covariance matrices for every class


### Import Libraries ###

In [6]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import sklearn.decomposition
from sklearn.model_selection import KFold

### Import Data ### 

In [5]:
# Read the training csv
train_pd = pd.read_csv('/home/caleb/Documents/Classes/machine-learning/assignments/assignment06/train.csv')

# Read the test csv
test_pd = pd.read_csv('/home/caleb/Documents/Classes/machine-learning/assignments/assignment06/test.csv')

# Values returns an array of pixel data from training set
values = train_pd.values[:,1:]

# Test values returns an array of pixel data from test set
test_values = test_pd.values

# response returns an array of labels
response = train_pd.values[:,0].ravel()

### Determine Optimal Number of Components ### 
 We can compute the optimal number of components by using the explained variance ratio as part of th `PCA` object

In [3]:
pca_obj = sklearn.decomposition.PCA()
x_trans = pca_obj.fit_transform(values)    

s = pca_obj.explained_variance_ratio_
sum=0.0
comp=0

for _ in s:
    sum += _
    comp += 1
    if(sum>=0.95):
        break

print("Number of eigenvectors to retain:", comp)

Number of eigenvectors to retain: 154


### Model Fitting ### 

In [4]:
# LDA 
sklearn_lda = LDA(n_components=comp)
# Taking in as second argument the Target as labels
training_LDA_2D = sklearn_lda.fit_transform(values, response)

/home/caleb/anaconda3/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


### Training Error ### 

In [72]:
print("Training Error:", sklearn_lda.score(values,response))

Training Error: 0.875214285714


## Cross -Validation Scores


In [65]:
# Recode the data 
X, y = values, response 

# Split the data into folds
n = len(X)
kf = KFold(n_splits = 20)
cross_error = []
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    skLDA = LDA(n_components = comp)
    clf = skLDA.fit_transform(X_train, y_train)
    score = skLDA.score(X_test, y_test)
    cross_error.append(score)

/home/caleb/anaconda3/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/caleb/anaconda3/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:455: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


In [70]:
print("mean of 20 cross validation errors:", np.array(cross_error).mean())
print("variance of 20 cross validation errors: ", np.array(cross_error).var())

mean of 20 cross validation errors: 0.865261904762
variance of 20 cross validation errors:  2.95572562358e-05


Low variance in the cross validation error is a good sign, however our error isn't as high as we'd like it to be. 

### Submission to Kaggle ###

In [112]:
Label = np.array(skLDA.predict(test_values))
ImageId = np.arange(start = 1, stop = Label.shape[0]+1)

In [114]:
df = pd.DataFrame({'ImageId': ImageId, 'Label': Label})
df.to_csv('LDA_submission.csv', index_label = False, index = False)

![title](LDA_Score.png)